In [27]:
import gurobipy as gp, pandas as pd
from collections import defaultdict

# ───────────────────────── 0. INPUT
T  = 10
failed_edges = {"e2","e3"}
max_wait = 2
w1, w2, w3 = 1000, 50, 10
CAPACITY = 3
BIG_M = 10**6
SINK = "SINK"                           # ★ 더미 종착 노드

edges = {
    "e1": ("n1","n2",1), "e2": ("n2","n3",1),
    "e3": ("n1","n4",2), "e4": ("n4","n3",2),
    "e5": ("n1","n5",2), "e6": ("n5","n6",1),
    "e7": ("n6","n7",1), "e8": ("n7","n3",1),
    "e9": ("n4","n2",1),
}

routes_nodes = {
    "T1":["n1","n2","n3"],
    "T2":["n1","n4","n3"],
    "T3":["n1","n5","n6","n7","n3"],
}
demand = {
    "T1":[("n1","n2",30),("n1","n3",50),("n2","n3",80)],
    "T2":[("n1","n4",50),("n1","n3",60),("n4","n3",10000)],
    "T3":[("n1","n5",800),("n1","n6",60),("n1","n7",75),
          ("n1","n3",10),("n5","n6",25),("n5","n7",35),
          ("n5","n3",70),("n6","n7",15),("n6","n3",25),
          ("n7","n3",35)],
}
trains = list(routes_nodes)

nodes = set(n for _, (s,d,_) in edges.items() for n in (s,d))
nodes.add(SINK)                          # ★ SINK 노드 포함

# ───────────────────────── 0-a. 시간확장 아크
arc_list = []                            # (from, to, eid, τ)

# 실제 이동 arc
for eid,(src,dst,tau) in edges.items():
    if eid in failed_edges: continue
    for t in range(T + 1 - tau):
        arc_list.append((f"{src}^{t}", f"{dst}^{t+tau}", eid, tau))

# wait-arc
for n in nodes - {SINK}:
    for w in range(1, max_wait+1):
        for t in range(T + 1 - w):
            arc_list.append((f"{n}^{t}", f"{n}^{t+w}", f"w_{n}_{w}", w))

# ★ dummy-arc : 후보 종착지 n^t → SINK^t  (τ=0)
for n in nodes - {SINK}:
    for t in range(T+1):
        arc_list.append((f"{n}^{t}", f"{SINK}^{t}", f"dummy_{n}", 0))

df_arc = pd.DataFrame(arc_list, columns=["from","to","eid","tau"])

# ───────────────────────── 0-b. 예정 도착시각
sched = {}
for tr, path in routes_nodes.items():
    t = 0; arr = {path[0]: 0}
    for i in range(len(path)-1):
        u,v = path[i], path[i+1]
        eid = next(e for e,(s,d,_) in edges.items() if s==u and d==v)
        t += edges[eid][2]
        arr[v] = t
    sched[tr] = arr

q_r = {tr: sum(q for *_,q in demand[tr]) for tr in trains}

# ───────────────────────── 1. 모델
m = gp.Model(); m.Params.OutputFlag = 0

# 1-a. 변수
x = {(tr, fr, to, eid): m.addVar(vtype=gp.GRB.BINARY)
     for (fr,to,eid,_) in arc_list for tr in trains}

h = {tr: m.addVar(vtype=gp.GRB.BINARY) for tr in trains}

r_T = {tr: set(routes_nodes[tr][1:]) for tr in trains}
y = {(tr,n): m.addVar(vtype=gp.GRB.BINARY)
     for tr in trains for n in r_T[tr]}

s = {(tr,n): m.addVar(vtype=gp.GRB.BINARY)
     for tr in trains for n in nodes - {SINK}}

z = {(tr,o,d): m.addVar(vtype=gp.GRB.BINARY)
     for tr in trains for (o,d,_) in demand[tr]}

delta = {(tr,o,d): m.addVar(lb=0)
         for tr in trains for (o,d,_) in demand[tr]}

t_arr = {(tr,n): m.addVar(lb=0,ub=T,vtype=gp.GRB.INTEGER)
         for tr in trains for n in nodes - {SINK}}

# ───────────────────────── 2. 제약
# (2) x ≤ h
for (tr,*_),var in x.items():
    m.addConstr(var <= h[tr])

# (3) 출발지 순유출
for tr in trains:
    r_o = routes_nodes[tr][0]
    out = gp.quicksum(v for (tr_,fr,*_),v in x.items()
                      if tr_==tr and fr.startswith(f"{r_o}^"))
    m.addConstr(out == h[tr])

# (4★) 종착 후보 선택 & dummy-arc 1개 연결
for tr in trains:
    m.addConstr(gp.quicksum(y[tr,n] for n in r_T[tr]) == h[tr])
    for n in r_T[tr]:
        dummy_sum = gp.quicksum(v for (tr_,fr,to,eid),v in x.items()
                                if tr_==tr and eid==f"dummy_{n}")
        m.addConstr(dummy_sum == y[tr,n])            # ★

# (5) 모든 노드( SINK 제외 ) 시간별 흐름보존
for tr in trains:
    r_o = routes_nodes[tr][0]
    for n in nodes - {SINK}:
        for t in range(T+1):
            nt = f"{n}^{t}"
            inflow  = gp.quicksum(v for (tr_,fr,to,*_),v in x.items()
                                  if tr_==tr and to == nt)
            outflow = gp.quicksum(v for (tr_,fr,to,*_),v in x.items()
                                  if tr_==tr and fr == nt)
            if n==r_o and t==0:        # 출발 노드·시각 = 순유출 = h
                m.addConstr(outflow - inflow == h[tr])
            elif n!=SINK:
                m.addConstr(inflow == outflow)

# (7’)(8’) 실제 in-arc 방문
for (tr, n), var_s in s.items():
    r_o = routes_nodes[tr][0]
    if n == r_o:
        real_flow = gp.quicksum(v for (tr_, fr, to, eid), v in x.items()
                                if tr_ == tr and fr.split("^")[0] == n
                                   and not eid.startswith(("w_", "dummy_")))
    else:
        real_flow = gp.quicksum(v for (tr_, fr, to, eid), v in x.items()
                                if tr_ == tr and to.split("^")[0] == n
                                   and not eid.startswith(("w_", "dummy_")))
    m.addConstr(real_flow >= var_s)
    m.addConstr(real_flow <= BIG_M * var_s)

# (10)~(12)+(15)
for tr in trains:
    for (o,d,_) in demand[tr]:
        m.addConstr(z[tr,o,d] <= s[tr,o])
        m.addConstr(z[tr,o,d] <= s[tr,d])
        m.addConstr(z[tr,o,d] >= s[tr,o] + s[tr,d] - 1)
        m.addConstr(z[tr,o,d] <= h[tr])

# 시간 링크 (dummy 제외)
for (tr,fr,to,eid),v in x.items():
    if eid.startswith(("w_","dummy_")): continue
    node_to, t_to = to.split("^"); t_to=int(t_to)
    m.addConstr(t_arr[tr,node_to] >= t_to - BIG_M*(1-v))
    m.addConstr(t_arr[tr,node_to] <= t_to + BIG_M*(1-v))

# Δ
for tr in trains:
    for (o,d,q) in demand[tr]:
        sched_t = sched[tr].get(d,T)
        m.addConstr(delta[tr,o,d] >= t_arr[tr,d]-sched_t - BIG_M*(1-z[tr,o,d]))
        m.addConstr(delta[tr,o,d] <= BIG_M*z[tr,o,d])

# (6) 용량 (dummy/wait 제외)
for eid,(src,dst,tau) in edges.items():
    if eid in failed_edges: continue
    for tt in range(T):
        flow = gp.quicksum(v for (tr,fr,to,e),v in x.items()
                           if e==eid and
                              int(fr.split("^")[1])<=tt<int(to.split("^")[1]))
        m.addConstr(flow <= CAPACITY)

# ───────────────────────── 3. 목적식
obj  = gp.quicksum(w1*q_r[tr]*(1-h[tr]) for tr in trains)
obj += gp.quicksum(w2*q*(h[tr]-z[tr,o,d])
                   for tr in trains for (o,d,q) in demand[tr])
obj += gp.quicksum(w3*q*delta[tr,o,d]*z[tr,o,d]
                   for tr in trains for (o,d,q) in demand[tr])
m.setObjective(obj, gp.GRB.MINIMIZE)

m.optimize()

# ───────────────────────── 4. 시간표 출력
rows=[]
for tr in trains:
    if h[tr].X<0.5:
        rows.append([tr,'-','-','-','-','Cancelled']); continue
    legs=[(int(fr.split("^")[1]),fr.split("^")[0],
           to.split("^")[0],int(to.split("^")[1]),eid)
          for (tr_,fr,to,eid),v in x.items()
          if tr_==tr and v.X>0.5 and not eid.startswith(("w_","dummy_"))]
    legs.sort(key=lambda k:k[0])
    for dep,frm,to,arr,eid in legs:
        rows.append([tr,frm,dep,to,arr,eid])

df=pd.DataFrame(rows,columns=["Train","From","Dep","To","Arr","Edge"])
print("\n=== Timetable (chronological) ===")
print(df.to_string(index=False))

# ───────────────────────── 5-A. objective breakdown
cancel_pen = sum(w1*q_r[tr]*(1-h[tr].X) for tr in trains)

unmet_pen  = sum(w2*q*(h[tr].X - z[tr,o,d].X)
                 for tr in trains for (o,d,q) in demand[tr])

delay_pen  = sum(w3*q*delta[tr,o,d].X*z[tr,o,d].X
                 for tr in trains for (o,d,q) in demand[tr])

print(f"\n=== Objective breakdown ===")
print(f"  Cancellation penalty : {cancel_pen:,.0f}")
print(f"  Un-served demand     : {unmet_pen:,.0f}")
print(f"  Delay penalty        : {delay_pen:,.0f}")
print(f"--------------------------------")
print(f"  Total objective      : {m.ObjVal:,.0f}")


# ───────────────────────── 5-B. train-level status
status_rows = []
for tr in trains:
    if h[tr].X < 0.5:
        status = "Cancelled"
    else:
        delayed = any(delta[tr,o,d].X > 1e-6 for (o,d,_) in demand[tr])
        unmet   = any(z[tr,o,d].X < 0.5 for (o,d,_) in demand[tr])
        if delayed:
            status = "Delayed"
        elif unmet:
            status = "Truncated/Rerouted"
        else:
            status = "On-time & complete"
    status_rows.append([tr, status])

df_stat = pd.DataFrame(status_rows, columns=["Train", "Status"])
print("\n=== Train status summary ===")
print(df_stat.to_string(index=False))



=== Timetable (chronological) ===
Train From  Dep To  Arr Edge
   T1   n1    0 n2    1   e1
   T2   n1    0 n5    2   e5
   T2   n5    2 n6    3   e6
   T2   n6    3 n7    4   e7
   T2   n7    4 n3    5   e8
   T3   n1    0 n5    2   e5
   T3   n5    2 n6    3   e6
   T3   n6    3 n7    4   e7
   T3   n7    4 n3    5   e8

=== Objective breakdown ===
  Cancellation penalty : 0
  Un-served demand     : 509,000
  Delay penalty        : 600
--------------------------------
  Total objective      : 509,600

=== Train status summary ===
Train             Status
   T1 Truncated/Rerouted
   T2            Delayed
   T3 On-time & complete
